# Data reading of the database
A.Iragena supervised by Mr Kaufmann

In [467]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import sqlite3 
from sqlite3 import Error
import csv

In [468]:
from striplog.striplog import Striplog
from striplog import Legend, Lexicon, Interval, Component, Position


In [469]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn
 

In [470]:
def select_datat_of_db(conn,table):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute(table)
 
    rows = cur.fetchall()
    #result = [dict(row) for row in cur.fetchall()]
    for row in rows:
        print(row)
    return(rows)
    #return(result)

In [471]:
def from_sqlite_Row_to_dict(list_with_rows):
    ''' Turn a list with sqlite3.Row objects into a dictionary'''
    d ={} # the dictionary to be filled with the row data and to be returned

    for i, row in enumerate(list_with_rows): # iterate throw the sqlite3.Row objects            
        l = [] # for each Row use a separate list
        for col in range(0, len(row)): # copy over the row date (ie. column data) to a list
            l.append(row[col])
        d[i] = l # add the list to the dictionary   
    return d

In [473]:
database = r"database.db"

### Create a database connection & import the data from the database

In [475]:
import pandas as pd

In [476]:
conn = create_connection(database)
bhs=pd.read_sql("SELECT * FROM Boreholes", conn)

In [477]:
tables = ['Boreholes', 'Component', 'Intervals', 'lexicon']
for i in tables:    
    print("---{table:s} data--- ".format(table=i))

---Boreholes data--- 
---Component data--- 
---Intervals data--- 
---lexicon data--- 


In [478]:
conn = create_connection(database)
tables = ['Boreholes', 'Component', 'Intervals', 'lexicon']
dfs = {}
with conn:    
    for i in tables:    
        print("---{table:s} data--- ".format(table=i))
        sql_cmd = "SELECT * FROM {table:s}".format(table=i) # to select the data of the table Boreholes
        dfs.update({i:  pd.read_sql(sql_cmd, conn)})

---Boreholes data--- 
---Component data--- 
---Intervals data--- 
---lexicon data--- 


In [479]:
dfs['Component'].query('Borehole=="F25" and Key=="color"')

,Borehole,Top,Base,Key,Value
1,F25,0.0,1.5,color,brun
3,F25,1.5,4.0,color,ocre
5,F25,4.0,6.0,color,gris
7,F25,0.0,1.5,color,brun
9,F25,1.5,4.0,color,ocre
11,F25,4.0,6.0,color,gris
13,F25,0.0,1.5,color,brun
15,F25,1.5,4.0,color,ocre
17,F25,4.0,6.0,color,gris
19,F25,0.0,1.5,color,brun


### Creating the striplog object

In [488]:
dfs['Intervals'].head()

,borehole,Top,Base,description
0,F10,0.0,1.5,remblais non-saturés
1,F10,1.5,4.0,remblais saturés
2,F10,4.0,6.0,alluvions
3,F11,0.0,1.5,remblais non-saturés
4,F11,1.5,4.0,remblais saturés


In [489]:
dfs['Component'].head()

,Borehole,Top,Base,Key,Value
0,F25,0.0,1.5,lithology,remblais
1,F25,0.0,1.5,color,brun
2,F25,1.5,4.0,lithology,remblais
3,F25,1.5,4.0,color,ocre
4,F25,4.0,6.0,lithology,silt


In [490]:
bh_test = 'F10'
dfs['Intervals'].query('borehole=="{borehole:s}"'.format(borehole=bh_test))

,borehole,Top,Base,description
0,F10,0.0,1.5,remblais non-saturés
1,F10,1.5,4.0,remblais saturés
2,F10,4.0,6.0,alluvions


In [491]:
bh_test = 'F10'
dfs['Intervals'].query('borehole=="{borehole:s}"'.format(borehole=bh_test))['Top']

0    0.0
1    1.5
2    4.0
Name: Top, dtype: float64

In [492]:
bh_test = 'F10'
bh_strip = []
intervals = dfs['Intervals'].query('borehole=="{borehole:s}"'.format(borehole=bh_test))[['Top','Base', 'description']]
for row in intervals.iterrows():
    components = dfs['Component'].query('Borehole=="{borehole:s}" and Top=={top:f} and Base=={base:f}'.format(borehole=bh_test, top=row[1]['Top'], base=row[1]['Base']))
    components_dict={}
    for r in components.iterrows():
        components_dict.update({r[1]['Key']:r[1]['Value']})
    bh_strip.append({'top': Position(middle=row[1]['Top'], units='m'), 
          'base': Position(middle=row[1]['Base'], units='m'),
          'description': row[1]['description'], 'components': [Component(components_dict)]})

In [493]:
for bh_row in dfs['Boreholes'].iterrows():
    print(bh_row[1][0])

F10
F11
F12
F13
F14
F15
F16
F20
F21
F22
F23
F24
F25


# Complete extraction of boreholes from the database

In [494]:
bhs = {}
for bh_row in dfs['Boreholes'].iterrows(): # For all bh
    bh_strip = []
    intervals = dfs['Intervals'].query('borehole=="{borehole:s}"'.format(borehole=bh_row[1][0]))[['Top','Base', 'description']] # Look for the interval for the bh
    for row in intervals.iterrows():
        components = dfs['Component'].query('Borehole=="{borehole:s}" and Top=={top:f} and Base=={base:f}'.format(borehole=bh_row[1][0], top=row[1]['Top'], base=row[1]['Base'])) # Look for components of the bh
        components_dict={}
        for r in components.iterrows():
            components_dict.update({r[1]['Key']:r[1]['Value']})
        bh_strip.append({'top': Position(middle=row[1]['Top'], units='m'), 
              'base': Position(middle=row[1]['Base'], units='m'),
              'description': row[1]['description'], 'components': [Component(components_dict)]})
    bhs.update({bh_row[1][0]: bh_strip})

In [495]:
bhs

{'F10': [{'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'units': 'm'}),
   'base': Position({'middle': 1.5, 'units': 'm'}),
   'description': 'remblais non-saturés',
   'components': [Component({})]},
  {'top': Position({'middle': 1.5, 'units': 'm'}),
   'base': Position({'middle': 4.0, 'units': 'm'}),
   'description': 'remblais saturés',
   'components': [Component({})]},
  {'top': Position({'middle': 4.0, 'units': 'm'}),
   'base': Position({'middle': 6.0, 'units': 'm'}),
   'description': 'alluvions',
   'components': [Component({})]}],
 'F11': [{'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'units': 'm'}),
   'base': Position({'middle': 1.5, 'units': 'm'}),
   'description': 'remblais non-saturés',
   'components': [Component({})]},
  {'top': Position({'middle': 1.5, 'units': 'm'}),
   'base': Position({'middle': 4.0, 'units': 'm'}),
   'description': 'remblais saturés',
   'components': [Component({})]},
  {'top': Position({'middle': 4.0, 'units': 'm'}

## Add lexicon

In [496]:
dfs['lexicon']

,Key,Value
0,color,brun
1,color,gris
2,color,ocre
3,lithology,remblais
4,lithology,silt


In [502]:
bh_strip.append(dfs['lexicon']) # Command to add smthg 

#### Structure of 1 interval of the striplog
{'top': Position({'middle': 51.24117647058824, 'units': 'm'}), 'base': Position({'middle': 54.81764705882354, 'units': 'm'}), 'description': '', 'data': {}, 'components': [Component({'lithology': 'sandstone', 'colour': 'grey', 'grainsize': 'vf-f'})]}


In [503]:
bh_strip


[{'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'units': 'm'}),
  'base': Position({'middle': 1.5, 'units': 'm'}),
  'description': 'remblais non-saturés',
  'components': [Component({'lithology': 'remblais', 'color': 'brun'})]},
 {'top': Position({'middle': 1.5, 'units': 'm'}),
  'base': Position({'middle': 4.0, 'units': 'm'}),
  'description': 'remblais saturés',
  'components': [Component({'lithology': 'remblais', 'color': 'ocre'})]},
 {'top': Position({'middle': 4.0, 'units': 'm'}),
  'base': Position({'middle': 6.0, 'units': 'm'}),
  'description': 'alluvions',
  'components': [Component({'lithology': 'silt', 'color': 'gris'})]},
          Key     Value
 0      color      brun
 1      color      gris
 2      color      ocre
 3  lithology  remblais
 4  lithology      silt]

In [499]:
d['F10', ] 


KeyError: ('F10',)

In [500]:
dic=from_sqlite_Row_to_dict(comp)


# Trials

In [ ]:
conn = create_connection(database)
with conn:    
        
    print("---BOREHOLES data--- ")
    table_bh="SELECT * FROM Boreholes" # to select the data of the table Boreholes
    bh=select_datat_of_db(conn,table_bh)
    
    print("---COMPONENTS data--- ")
    table_comp="SELECT * FROM Component" # to select the data of the table component
    comp=select_datat_of_db(conn,table_comp)
    
    print("---INTERVAL data--- ")
    table_int="SELECT * FROM Intervals" # to select the data of the table Intervals
    intvl=select_datat_of_db(conn,table_int)
    
    print("---LEXICON--- ")
    table_lex="SELECT * FROM lexicon" # to select the data of the table Intervals
    lex=select_datat_of_db(conn,table_lex)
    
    
 

In [ ]:
conn = create_connection(database)
with conn:    
        
    print("---BOREHOLES data--- ")
    table_bh="SELECT * FROM Boreholes" # to select the data of the table Boreholes
    bh=select_datat_of_db(conn,table_bh)
    
    print("---COMPONENTS data--- ")
    table_comp="SELECT * FROM Component" # to select the data of the table component
    comp=select_datat_of_db(conn,table_comp)
    
    print("---INTERVAL data--- ")
    table_int="SELECT * FROM Intervals" # to select the data of the table Intervals
    intvl=select_datat_of_db(conn,table_int)
    
    print("---LEXICON--- ")
    table_lex="SELECT * FROM lexicon" # to select the data of the table Intervals
    lex=select_datat_of_db(conn,table_lex)
 
 

In [ ]:
bh # is a list, cause it has brackets
bh[0]
lex
type(intvl)
intvl[0][0]
bh[1][0]
comp[1][4]
lexicon=lex